In [ ]:
%pip install -r requirements.txt -Uq

In [ ]:
# import the class modules that support the Text to SQL pipeline
import app_constants as app_consts
import identity_service_facade
import llm_facade
import rdbms_facade

import pre_process_request
import prepare_request
import test_cases

In [ ]:
# Instantiate objects for the classes that will be used
test_user_requests = test_cases.TestCases()
llm_service_facade = llm_facade.LlmFacade(app_consts.BEDROCK_MODEL_ID)
id_service_facade = identity_service_facade.IdentityServiceFacade()
rdbms_service_facade = rdbms_facade.RdbmsFacade()

request_pre_processor = pre_process_request.PreProcessRequest(llm_service_facade)
request_preparer = prepare_request.PrepareRequest()

### Pull an example user request from the test set

In [ ]:
user_request = test_user_requests.get_test_case(4)
user_request

### Step 1 - Pre-process the use request, determine their domain, and if there are identifiers that need to be mapped to IDs

In [ ]:
pre_processed_request = request_pre_processor.run(user_request)
domain = pre_processed_request[app_consts.DOMAIN]
pre_processed_request

### Step 2 - Map identifiers/references to database IDs

In [ ]:
named_resources = pre_processed_request[app_consts.NAMED_RESOURCES]
if len(named_resources) > 0:
    identifiers = id_service_facade.resolve(named_resources)
    # add identifies to the pre_processed_request object
    pre_processed_request[app_consts.IDENTIFIERS] = identifiers
else:
    pre_processed_request[app_consts.IDENTIFIERS] = []
pre_processed_request

### Step 3 - Prepare the prompt for LLM and generate SQL preamable if neccessary

In [ ]:
prepared_request = request_preparer.run(pre_processed_request)
prepared_request

### Step 4 - Call the LLM to generate the SQL

In [ ]:
llm_response = llm_service_facade.invoke(prepared_request[app_consts.LLM_PROMPT])
generated_sql = llm_response[app_consts.LLM_OUTPUT]
generated_sql

### Step 5 - Run the generated SQL against the RDBMS

In [ ]:
sql_script = prepared_request[app_consts.SQL_PREAMBLE] + [generated_sql[app_consts.SQL]]
database = app_consts.get_database_for_domain(domain)
results = rdbms_service_facade.execute_sql(database, sql_script)
results